In [47]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
import random

In [48]:
words =  open('names.txt','r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [49]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [50]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182499, 3]) torch.Size([182499])
torch.Size([22852, 3]) torch.Size([22852])
torch.Size([22795, 3]) torch.Size([22795])


In [51]:
#MLP Structure
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP
#fan-in = n_embd * block_size fan-out = n_hidden
C  = torch.randn((vocab_size, n_embd)) ##idea is to scale w such that resulting data follow normal gaussian dist.v
W1 = torch.randn((n_embd * block_size, n_hidden)) * (5/3)/((n_embd * block_size)**0.5) #multiply by gain, divide by sqrt of fan in (# of inputs)
#b1 = torch.randn((n_hidden)) * 0.01 #ensuring tanh doesn't create dead neurons by lowering first pass
W2 = torch.randn((n_hidden, vocab_size)) *0.01 #reducing size of logits to reduce initial loss
b2 = torch.randn((vocab_size)) * 0 #dont want to add random numbers (high initial loss)

bngain = torch.ones((1,n_hidden)) #note: one side effect of batchnorm is each h value coming through gets influenced by its batchmates,
bnbias = torch.zeros((1,n_hidden)) #cont.: this isn't bad, slightly changes inputs (creates more noise), and actually fights overfitting (not modern anymore)
bnmean_running = torch.zeros((1,n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

#Note: deep NNs often have the structure linear layer (W), normalization (BatchNorm), nonlinearity (Tanh)

12097


In [ ]:
#Optimization
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
     
    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    hpreact = embcat @ W1 #+ b1 # hidden layer pre-activation # b1 doesn't have influence b/c of BN 

        #batch normalization: #divide each batch by std and subtract mean to create Gaussian at init!!
        #add in gain and bias such that NN can move this dist. as it learns, common to add after linear layers (not all)
    bnmeani = hpreact.mean(0, keepdim = True)
    bnstdi = hpreact.std(0, keepdim = True)
    hpreact = bngain * (hpreact - bnmeani)/ bnstdi + bnbias
    
    with torch.no_grad():
        bnmean_running = 0.999 *bnmean_running + 0.001 *bnmeani
        bnstd_running = 0.999 *bnstd_running + 0.001 *bnstdi
    
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
      
      # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
    
      # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    #break

      0/ 200000: 3.3071
  10000/ 200000: 2.0988
  20000/ 200000: 1.9241
  30000/ 200000: 1.9332
  40000/ 200000: 1.8161
  50000/ 200000: 2.1702
  60000/ 200000: 1.9482
  70000/ 200000: 2.1338
  80000/ 200000: 2.3519
  90000/ 200000: 1.9273
 100000/ 200000: 2.4621
 110000/ 200000: 1.8724
 120000/ 200000: 2.2350
 130000/ 200000: 1.7278
 140000/ 200000: 2.2601


In [ ]:
plt.figure(figsize = (20,10)) #looking at Tanh activations, looking for dead neurons
plt.imshow(h.abs()> 0.99, cmap = 'gray', interpolation = 'nearest') #black = active

In [ ]:
plt.hist(h.view(-1).tolist(), 50); #inputs into backpropagation (1-T**2) * p.grad
# the outputs of tanh

__What was wrong^:__
originially, network was initalizing at way too high of loss (~25), we expect ~3.5 to be initial loss (reflects nll of 1/27). Happening because the initial probability distributions are not uniform (randomly generated).

In [ ]:
plt.plot(lossi)

In [ ]:
# batchnorm expects a batch input, but for one input, calculate global mean and std:
# can also be done during training
with torch.no_grad():
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 #+ b1
    bnmean = hpreact.mean(0, keepdims = True) #could also estimate these during training
    bnstd = hpreact.std(0, keepdims = True)

In [ ]:
@torch.no_grad() #disables gradient tracking for efficiency
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 #+b1
    #hpreact = bngain * (hpreact -bnmean) /bnstd +bnbias
    hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
split_loss('train')
split_loss('val')

In [ ]:
#Sampling from model:
for _ in range(20):
    out = []
    context = [0] *block_size
    while True:
        emb = C[torch.tensor([context])]
        embcat = emb.view(emb.shape[0], -1)
        hpreact = embcat @ W1 #+b1
        #hpreact = bngain * (hpreact -bnmean) /bnstd +bnbias
        hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
        h = torch.tanh(hpreact)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim = 1)
        ix = torch.multinomial(probs, num_samples = 1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

In [ ]:
#REwriting PyTorch module--------------------
#rewrite of PyTorch nn section

class Linear:
    def __init__(self, fan_in, fan_out, bias = True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
        
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    def parameters(self):
        return [self.weight] +([] if self.bias is None else [self.bias])

class BatchNorm1d:
    def __init__(self, dim, eps = 1e-5, momentum = 0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        #params:
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        #buffers:
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
        
    def __call__(self,x):
        if self.training:
            xmean = x.mean(0, keepdim= True)
            xvar = x.var(0, keepdim = True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x-xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self,x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []

n_embd = 10 #dimensionality of embedding space
n_hidden = 100 #number of neurons in hidden layer

C = torch.randn((vocab_size, n_embd))
layers = [ #introducing batchnorm allows for gain to be much less brittle
    Linear(n_embd * block_size, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size), BatchNorm1d(vocab_size)
]

gain = 5/3
with torch.no_grad():
    #make last layer less confident:
    layers[-1].gamma *=0.1
    #layers[-1].weight *= 0.1 for linear final layer
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight*= gain #adding gain to all linear layers to prevent data from being squashed too much by Tanh
parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True   

In [ ]:
#Optimization
max_steps = 200000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):
  
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
     
    # forward pass
    emb = C[Xb] # embed the characters into vectors
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)

    # backward pass
    for layer in layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()
      
      # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
    
      # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    with torch.no_grad():
        ud.append([(lr*p.grad.std()/p.data.std()).log10().item() for p in parameters])
    if i >1000:
        break

In [ ]:
# visualizing histograms for each tanh activation layer
#activations stay relatively uniform (not too extreme)
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out
    print('layer %d (%10s): mean %+.2f, std %.2f, saturated: %.2f%%' % (i, layer.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean()*100))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('activation distribution')

In [ ]:
# visualizing histogram of gradient distribution for tanh layers
#gradients stay roughly the same accross all layers
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out.grad
    print('layer %d (%10s): mean %+f, std %e' % (i, layer.__class__.__name__, t.mean(), t.std()))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('gradient distribution')

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i,p in enumerate(parameters):
  t = p.grad
  if p.ndim == 2:
    print('weight %10s | mean %+f | std %e | grad:data ratio %e' % (tuple(p.shape), t.mean(), t.std(), t.std() / p.std()))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'{i} {tuple(p.shape)}')
plt.legend(legends)
plt.title('weights gradient distribution');

In [ ]:
#visualizing update vs. data sizes, ratio should be around -3 and evening out
#can reveal if lr is too low/high
plt.figure(figsize=(20, 4))
legends = []
for i,p in enumerate(parameters):
  if p.ndim == 2:
    plt.plot([ud[j][i] for j in range(len(ud))])
    legends.append('param %d' % i)
plt.plot([0, len(ud)], [-3, -3], 'k') # these ratios should be ~1e-3, indicate on plot
plt.legend(legends);